In [3]:
# from pyspark import SparkConf, SparkContext
# from pyspark.sql import functions as F
# from pyspark.sql import SQLContext
# from pyspark.sql import SparkSession
import spacy

In [4]:
nlp = spacy.load('en_core_web_lg')

OSError: [E050] Can't find model 'en_core_web_lg'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
# spark = SparkSession.builder\
#     .master('local[*]')\
#     .appName('explore')\
#     .getOrCreate()
# sc = spark.sparkContext._conf.setAll([('spark.driver.maxResultSize', '8g')])

In [1]:
processed_df = spark.read.json('/common/users/shared/cs543_fall22_group3/combined/combined_raw')
processed_df.show()

+--------------------+
|       selected_text|
+--------------------+
|University of Hyd...|
|K Chandrasekhar R...|
|Temple town Palit...|
|UDF candidate C R...|
|Greater Hyderabad...|
|Corporator; other...|
|8 lions in Hydera...|
|19% brought dead;...|
|Telangana: 'All I...|
|Left to own devic...|
|Telangana: Daily ...|
|Cases dip for 2nd...|
|Kerala assembly e...|
|High court notice...|
|India and Europe ...|
|Heavy rains lash ...|
|Priest attempts s...|
|Deputy speaker as...|
|Rain; hail hit ke...|
|Mumbai: CBI probe...|
+--------------------+
only showing top 20 rows



In [ ]:
processed_df.count()

In [ ]:
def vectorize(text):
    return nlp(str(text)).vector

vectorize_udf = F.udf(lambda x: vectorize(x))
processed_df = processed_df.withColumn('vector', vectorize_udf(F.col('selected_text')))

In [ ]:
processed_df.write.mode('Overwrite').json('/common/users/shared/cs543_fall22_group3/combined/deep_vectors')